Imports. The output of `tf.__version__` should read something like 2.#

In [1]:
import os
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import pycocotools
import tensorflow as tf
from absl import flags
from pathlib import Path
print(tf.__version__)
import logging
logging.basicConfig(level=logging.INFO)
from tensorflow.python.client import device_lib
for d in device_lib.list_local_devices():
    print(d)
    print('---')

2.6.0
name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10290344880804211473

---
name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7769554944
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16889670329778467687
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1"

---


2021-09-26 22:54:29.778890: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-26 22:54:29.954846: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1885] Ignoring visible gpu device (device: 1, name: NVS 510, pci bus id: 0000:03:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.
2021-09-26 22:54:30.386450: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 7409 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


Set directories.

In [2]:
DATA_DIR = Path("./data/10k")
TRAIN_FILE = DATA_DIR / "10k_train.tfrecord"
TEST_FILE = DATA_DIR / "10k_test.tfrecord"
CLASS_LABELS_FILE = DATA_DIR / "classlabels.txt"
CKPT_DIR = Path("./checkpoints") / "ckpts06"
EXPORT_DIR = Path("./export/") / "06"
os.makedirs(CKPT_DIR, exist_ok=True)
os.makedirs(EXPORT_DIR, exist_ok=True)

print("Reading data from ", DATA_DIR.resolve())
print("Saving Checkpoints to ", CKPT_DIR.resolve())
print("Exporting to ", EXPORT_DIR.resolve())

Reading data from  /home/ms/prg/train_targetID/data/10k
Saving Checkpoints to  /home/ms/prg/train_targetID/checkpoints/ckpts06
Exporting to  /home/ms/prg/train_targetID/export/06


Read class labels.

In [3]:
with open(CLASS_LABELS_FILE, "r") as f:
    label_map = [s.strip() for s in f.readlines()]
    
label_map_dict = {}
for i, l in enumerate(label_map):
    label_map_dict[i+1] = l

for k, v in label_map_dict.items():
    print(k, ": ", v)

1 :  circle
2 :  diamond
3 :  halfcircle
4 :  heart
5 :  hexagon
6 :  hexstar
7 :  moon
8 :  octagon
9 :  pentagon
10 :  pentstar
11 :  plus
12 :  square
13 :  trapezoid
14 :  triangle


In [4]:
train_data = object_detector.DataLoader(
    tfrecord_file_patten = str(TRAIN_FILE.resolve()),
    size = 9000,
    label_map = label_map_dict
)

test_data = object_detector.DataLoader(
    tfrecord_file_patten = str(TEST_FILE.resolve()),
    size = 1000,
    label_map = label_map_dict
)

Create spec with hyperparameters.

In [5]:
hparams = {
    "map_freq" : 1,
    "learning_rate" : 0.2,
    "jitter_max" : 0.5,
    "jitter_min" : 0.05,
}
spec = object_detector.EfficientDetLite4Spec(
    hparams=hparams, model_dir=CKPT_DIR, epochs=500, batch_size=16,
    var_freeze_expr='(efficientnet|fpn_cells|resample_p6)',
    verbose=1
)

2021-09-26 22:54:30.442692: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1885] Ignoring visible gpu device (device: 1, name: NVS 510, pci bus id: 0000:03:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.
2021-09-26 22:54:30.445746: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7409 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


Train model

In [ ]:
model = object_detector.create(
    train_data = train_data,
    model_spec = spec,
    validation_data = test_data,
    train_whole_model = False,
    do_train = True,
)
model.export(
    export_dir = EXPORT_DIR,
    export_format = [ExportFormat.TFLITE, ExportFormat.LABEL, ExportFormat.SAVED_MODEL]
)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...
INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:target_size = (640, 640), output_size = (640, 640)
INFO:absl:LR schedule method: cosine
INFO:absl:Use SGD optimizer
2021-09-26 22:54:53.485918: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/500


2021-09-26 22:55:18.245071: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204
2021-09-26 22:55:20.746668: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.96GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-09-26 22:55:21.050438: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.76GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-09-26 22:55:21.065584: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.56GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory

562/562 [==============================] - 827s 1s/step - det_loss: 1.2188 - cls_loss: 0.8807 - box_loss: 0.0068 - reg_l2_loss: 0.0098 - loss: 1.2286 - learning_rate: 0.0290 - gradient_norm: 1.4820 - val_det_loss: 1.1690 - val_cls_loss: 0.9931 - val_box_loss: 0.0035 - val_reg_l2_loss: 0.0109 - val_loss: 1.1800

Epoch 00001: saving model to checkpoints/ckpts06/ckpt-1
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

562/562 [==============================] - 804s 1s/step - det_loss: 0.4235 - cls_loss: 0.2982 - box_loss: 0.0025 - reg_l2_loss: 0.0510 - loss: 0.4745 - learning_rate: 0.0038 - gradient_norm: 1.2004 - val_det_loss: 0.7024 - val_cls_loss: 0.6005 - val_box_loss: 0.0020 - val_reg_l2_loss: 0.0509 - val_loss: 0.7533

Epoch 00411: saving model to checkpoints/ckpts06/ckpt-411
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(99200, 7)
0/99200
DONE (t=0.34s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.03s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.375
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.469
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.434
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.119
 Average Preci

562/562 [==============================] - 802s 1s/step - det_loss: 0.4305 - cls_loss: 0.3024 - box_loss: 0.0026 - reg_l2_loss: 0.0507 - loss: 0.4812 - learning_rate: 0.0034 - gradient_norm: 1.2120 - val_det_loss: 0.6922 - val_cls_loss: 0.5850 - val_box_loss: 0.0021 - val_reg_l2_loss: 0.0507 - val_loss: 0.7429

Epoch 00416: saving model to checkpoints/ckpts06/ckpt-416
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(99200, 7)
0/99200
DONE (t=0.95s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.99s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.378
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.472
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.438
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.122
 Average Preci

562/562 [==============================] - 803s 1s/step - det_loss: 0.4159 - cls_loss: 0.2929 - box_loss: 0.0025 - reg_l2_loss: 0.0505 - loss: 0.4664 - learning_rate: 0.0030 - gradient_norm: 1.1858 - val_det_loss: 0.7026 - val_cls_loss: 0.5978 - val_box_loss: 0.0021 - val_reg_l2_loss: 0.0504 - val_loss: 0.7531

Epoch 00421: saving model to checkpoints/ckpts06/ckpt-421
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(99200, 7)
0/99200
DONE (t=0.94s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.13s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.389
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.480
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.444
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.126
 Average Preci

562/562 [==============================] - 801s 1s/step - det_loss: 0.4206 - cls_loss: 0.2961 - box_loss: 0.0025 - reg_l2_loss: 0.0502 - loss: 0.4709 - learning_rate: 0.0026 - gradient_norm: 1.2411 - val_det_loss: 0.6781 - val_cls_loss: 0.5751 - val_box_loss: 0.0021 - val_reg_l2_loss: 0.0502 - val_loss: 0.7283

Epoch 00426: saving model to checkpoints/ckpts06/ckpt-426
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(99200, 7)
0/99200
DONE (t=0.30s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.97s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.393
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.482
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.449
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.130
 Average Preci

562/562 [==============================] - 801s 1s/step - det_loss: 0.4275 - cls_loss: 0.2993 - box_loss: 0.0026 - reg_l2_loss: 0.0501 - loss: 0.4775 - learning_rate: 0.0023 - gradient_norm: 1.2370 - val_det_loss: 0.6803 - val_cls_loss: 0.5750 - val_box_loss: 0.0021 - val_reg_l2_loss: 0.0500 - val_loss: 0.7304

Epoch 00431: saving model to checkpoints/ckpts06/ckpt-431
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(99200, 7)
0/99200
DONE (t=0.34s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.01s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.386
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.479
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.439
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.123
 Average Preci